# PCA and Clustering

In [1]:
import numpy as np
import pandas as pd
pd.pandas.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
import seaborn as sns
import pyreadr
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

ModuleNotFoundError: No module named 'pyreadr'

#### Loading the preprocessed 2022 and 2023 data's

In [ ]:
dataset_2023 = pd.read_csv('datasets/data_2023_preprocessed.csv')
dataset_2022 = pd.read_csv('datasets/data_2022_preprocessed.csv')

In [ ]:
dataset_2023

In [ ]:
# double checking again low variance features
# applying VarianceThreshold to filter low-variance features (threshold=0.8)
selector = VarianceThreshold(threshold=0.01)
selector.fit(dataset_2023)

# get boolean mask of retained features
selected_features_mask = selector.get_support()

# get the names of low-variance features
low_variance_features = (dataset_2023.columns[~selected_features_mask]).tolist()
print(low_variance_features)

In [ ]:
print(len(low_variance_features))

In [ ]:
# after removing the low variance features after double checking

dataset_2022_remv_low_var = dataset_2022.drop(low_variance_features, axis=1)
dataset_2023_remv_low_var = dataset_2023.drop(low_variance_features, axis=1)
dataset_2023_remv_low_var

In [ ]:
# removing DON'T KNOW, BLANK (not answered and not asked questions)
# the standard code based on the codebook
refused_answers = [97, 997, 9997]
blank_answers = [99, 999, 9999 ] 
dont_know_answers = [ 94, 994, 9994]

# function to rem
def remove_unwanted_answers(data, answers):
    """
    remove rows from a data where any column contains values from the answers list.
    """
    # creat a mask to identify unwanted answers
    mask = data.isin(answers).any(axis=1)
    # Invert the mask to keep rows without refused answers
    data_filtered = data[~mask]
    print(f"Removed {len(data) - len(data_filtered)} rows containing unnecessary answers.")
    return data_filtered.reset_index(drop=True)

def remove_columns_with_refused_answers(data, refused_answers, threshold):
    """
    remove columns from a dataFrame where more than a specified percentage of values are (refused_answers).
    """
    # cal. the percentage of refused answers in each column
    refused_percentage = data.isin(refused_answers).mean()
    columns_to_drop = refused_percentage[refused_percentage > threshold].index
    data_filtered = data.drop(columns=columns_to_drop)
    print(f"Removed {len(columns_to_drop)} columns with more than {threshold * 100}% blank answers.")
    return data_filtered

In [ ]:
# data_filtered_2023 = remove_columns_with_refused_answers(dataset_2022_remv_low_var, blank_answers, 0.09)
# data_filtered_2023 

In [ ]:
# so removing the rows for blank answer,dont_know_answers doesn't show reliable performance when clustering bas
data_filtered_2023 = remove_columns_with_refused_answers(dataset_2022_remv_low_var )



#### Scaling(Normalizing)- Centering the data

In [ ]:
# scaling the data 

def standardscaling(data):
    original_data = data.copy()
    # numerical_data = data.select_dtypes(include=['float64', 'int64'])  # all data are numerical at the moment
    scaler = StandardScaler()
    scaled_numerical_data = scaler.fit_transform(data)
    scaled_data = pd.DataFrame(scaled_numerical_data, columns=original_data.columns)
    return scaled_data

In [ ]:
scaled_data_2022 = standardscaling(dataset_2022_remv_low_var)
scaled_data_2023 = standardscaling (dataset_2023_remv_low_var)
scaled_data_2023

### Outlier Removal

In [ ]:
scaled_data_2023

- For high-dimensional data, algorithmic methods like Isolation Forest or Local Outlier Factor can detect outliers by learning the data’s structure. “One efficient way of performing outlier detection in high-dimensional datasets is to use random forests. The IsolationForest isolates observations by randomly selecting a feature and split value”

In [ ]:
# IsolationForest- Outlier Removal
from sklearn.ensemble import IsolationForest

iso = IsolationForest(contamination=0.01, random_state=42) # initially 0.01
outlier_preds = iso.fit_predict(scaled_data_2023)  # -1 for outliers, 1 for inliers
X_inliers = scaled_data_2023[outlier_preds == 1]
X_inliers 

## PCA

#### 1. Centering the data

In [ ]:
# centering the data  -- similar to the scaling we did above so skip
def centering_data ( data):
    centered_data = data - data.mean()
    return centered_data

In [ ]:
centered_data_2023 = centering_data(X_inliers)
centered_data_2023

In [ ]:
centered_data_2023

##### Decompoisition

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)  # keep 95% of variance
X_pca = pca.fit_transform(scaled_data_2023)
print(pca.n_components_)      # number of components selected
print(pca.explained_variance_ratio_)  # variance explained by each component


In [ ]:
# applying PCA to compute the explained variance for each component
# pca = PCA()
# pca.fit(centered_data_2023)

# the scree plot (explained variance vs. number of components)
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, marker='o', linestyle='--', color='red')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
### Based on the scree plot elbow it suggest to use 30 to componennts

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_.cumsum(), marker='o', linestyle='--', color='blue')
plt.title('Cumulative Explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.axhline(y=0.85, color='r', linestyle='--', label='85% threshold')

plt.grid(True)
plt.show()

# num_components_95 = np.argmax(cumulative_variance >= 0.85) + 1
# print(f"Components needed for 85% variance: {num_components_95}")

In [ ]:
# apply PCA with the selected number of components
pca = PCA(n_components=40)
pca_result = pca.fit_transform(centered_data_2023)

# visualize the first two principal components
plt.figure(figsize=(10, 6))
plt.scatter(pca_result[:, 0], pca_result[:, 3], alpha=0.5, s=10, color='blue')
plt.title('PCA - First Two Principal Components')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.show()

# # check the distribution of the first few principal components
# # Distribution of PC1
# plt.figure(figsize=(10, 6))
# plt.hist(pca_result[:, 0], bins=50, color='skyblue', edgecolor='black')
# plt.title('Distribution of Principal Component 1')
# plt.xlabel('Principal Component 1')
# plt.ylabel('Frequency')
# plt.grid(True)
# plt.show()

In [ ]:
pca = PCA(n_components=0.80)
pca_result = pca.fit_transform(centered_data_2023)
print("Explained variance by PC1:", pca.explained_variance_ratio_[0])
print("Explained variance by PC2:", pca.explained_variance_ratio_[1])

### ICA

In [ ]:
from sklearn.decomposition import FastICA

ica = FastICA()
X_ica = ica.fit_transform(centered_data_2023)  # Independent Components

plt.scatter(X_ica[:, 0], X_ica[:, 1], alpha=0.5)
plt.xlabel("Independent Component 1")
plt.ylabel("Independent Component 2")
plt.title("ICA - First Two Independent Components")
plt.grid()
plt.show()

In [ ]:
plt.scatter(X_ica[:, 0], X_ica[:, 2], alpha=0.5, color='blue')
plt.xlabel("Independent Component 1")
plt.ylabel("Independent Component 2")
plt.title("ICA - First Two Independent Components")
plt.grid()
plt.show()

## Clustering- With PCA

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


n_clusters = 2 
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(pca_result[:, :50])  # Using the first 3 PCs for clustering
labels = kmeans.labels_

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(pca_result[:, 0], pca_result[:, 1], pca_result[:, 2], c=labels, cmap='viridis', s=30, alpha=0.5)

plt.title('K-Means Clustering (First 3 Principal Components)')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
plt.colorbar(scatter, label='Cluster Label')
plt.show()

silhouette_avg = silhouette_score(pca_result[:, :3], labels)
print(f'Silhouette Score for {n_clusters} clusters: {silhouette_avg}')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

n_clusters = 2 
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(pca_result[:, :40])  # Using the first 3 PCs for clustering
labels = kmeans.labels_

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(pca_result[:, 0], pca_result[:, 1], pca_result[:, 2], c=labels, cmap='viridis', s=30, alpha=0.5)

plt.title('K-Means Clustering (First 3 Principal Components)')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
plt.colorbar(scatter, label='Cluster Label')
plt.show()

silhouette_avg = silhouette_score(pca_result[:, :3], labels)
print(f'Silhouette Score for {n_clusters} clusters: {silhouette_avg}')

In [ ]:
a = []
b = []
for i in labels.tolist():
    if i==0:
        a.append(i)
    else:
         b.append(i)
print(len(a))
print(len(b))

In [ ]:
# adding the cluster label to the original data
dataset_2023_2023_indexed = dataset_2023_remv_low_var.loc[centered_data_2023.index] 
dataset_2023_2023_indexed

In [ ]:
dataset_2023_2023_indexed['CLUSTER'] = labels
dataset_2023_2023_indexed

In [ ]:
dataset_2023_2023_indexed.to_csv('dataseeet_2023_with_cluster_label_second.csv', index=False)

## Clustering-with ICA

In [ ]:
n_clusters = 2 

kmeans_ica = KMeans(n_clusters=n_clusters, random_state=42)
kmeans_ica.fit(X_ica[:, :296])  

labels = kmeans_ica.labels_
labels

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(X_ica[:, 0], X_ica[:, 1], X_ica[:, 2], c=labels, cmap='viridis', s=30, alpha=0.5)

plt.title('K-Means Clustering (First 3 Principal Components)')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
plt.colorbar(scatter, label='Cluster Label')
plt.show()

silhouette_avg = silhouette_score(pca_result[:, :3], labels)
print(f'Silhouette Score for {n_clusters} clusters: {silhouette_avg}')


In [ ]:
# Conclusion 
# - The clustering outperforms  with PCA compare to  ICA
